<a href="https://colab.research.google.com/github/tamojeetroychowdhury/Facial-Expression-Recognition/blob/main/Week-2/FER_DeX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
from PIL import Image

In [3]:
import numpy as np

In [5]:
np.array([1,2,3,4,5,6]).reshape(3,2)

array([[1, 2],
       [3, 4],
       [5, 6]])

In [19]:
images = []
labels = []

for i in range(1,84):
  img = Image.open(f'gdrive/My Drive/FER/CK+/surprise/SU ({i}).png').convert('L')
  try:
    data = np.array(img.getdata()).reshape((490,640))
    images.append([data])
    labels.append(0)
  except:
    continue


for i in range(1,70):
  img = Image.open(f'gdrive/My Drive/FER/CK+/happiness/H ({i}).png').convert('L')
  try:
    data = np.array(img.getdata()).reshape((490,640))
    images.append([data])
    labels.append(1)
  except:
    continue


for i in range(1,26):
  img = Image.open(f'gdrive/My Drive/FER/CK+/fear/F ({i}).png').convert('L')
  try:
    data = np.array(img.getdata()).reshape((490,640))
    images.append([data])
    labels.append(2)
  except:
    continue


for i in range(1,60):
  img = Image.open(f'gdrive/My Drive/FER/CK+/disgust/D ({i}).png').convert('L')
  try:
    data = np.array(img.getdata()).reshape((490,640))
    images.append([data])
    labels.append(3)
  except:
    continue


for i in range(1,594):
  img = Image.open(f'gdrive/My Drive/FER/CK+/neutral/N ({i}).png').convert('L')
  try:
    data = np.array(img.getdata()).reshape((490,640))
    images.append([data])
    labels.append(4)
  except:
    continue


for i in range(1,29):
  img = Image.open(f'gdrive/My Drive/FER/CK+/sadness/S ({i}).png').convert('L')
  try:
    data = np.array(img.getdata()).reshape((490,640))
    images.append([data])
    labels.append(5)
  except:
    continue


for i in range(1,46):
  img = Image.open(f'gdrive/My Drive/FER/CK+/anger/A ({i}).png').convert('L')
  try:
    data = np.array(img.getdata()).reshape((490,640))
    images.append([data])
    labels.append(6)
  except:
    continue

In [18]:
len(images)

491

In [20]:
np.array(images).shape

(845, 1, 490, 640)

In [21]:
shuffler = [[images[i], labels[i]] for i in range(845)]

In [25]:
np.random.shuffle(shuffler)

In [27]:
images_new = [shuffler[i][0] for i in range(845)]
labels_new = [shuffler[i][1] for i in range(845)]

In [28]:
np.array(labels_new).shape

(845,)

In [29]:
import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

#df = pd.read_csv('gdrive/My Drive/fer2013.csv')

In [30]:
images_new = np.array(images_new, dtype = np.float32)
dataset = torch.tensor(images_new)

In [31]:
dataset.shape

torch.Size([845, 1, 490, 640])

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [32]:
input_size = 2304
hidden_size = 500 
num_classes = 10
#num_epochs = 3
#batch_size = 2000
learning_rate = 0.01

b = 20

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, 7, stride = 2, padding = 3)
        self.pool1 = nn.MaxPool2d(3, 2)
        self.lrn1 = nn.LocalResponseNorm(64)

        self.conv2a = nn.Conv2d(64, 96, 1)
        self.conv2b = nn.Conv2d(96, 208, 3, padding = 1)
        self.pool2a = nn.MaxPool2d(3, 1, padding = 1)
        self.conv2c = nn.Conv2d(64, 64, 1)
        #self.cat2 = torch.cat()
        self.pool2b = nn.MaxPool2d(3, 2)

        self.conv3a = nn.Conv2d(272, 96, 1)
        self.conv3c = nn.Conv2d(272, 64, 1)

        self.fc1 = nn.Linear(6152640//b, 90)
        self.fc3 = nn.Linear(90, 7)

        #self.lkr = nn.LeakyReLU()

    def forward(self, x):
        x = self.lrn1(self.pool1(F.relu(self.conv1(x))))

        y = F.relu(self.conv2c(self.pool2a(x)))
        z = F.relu(self.conv2b(F.relu(self.conv2a(x))))
        x = self.pool2b(torch.cat((y,z),1))

        y = F.relu(self.conv3c(self.pool2a(x)))
        z = F.relu(self.conv2b(F.relu(self.conv3a(x))))
        x = self.pool2b(torch.cat((y,z),1))

        x = x.view(-1, 6152640//b)
        x = F.relu(self.fc3(F.relu(self.fc1(x))))
        return x



model = NeuralNet()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [33]:
labels_new = torch.tensor(labels_new)

In [35]:
batch_size = b
num_epochs = 4

for epoch in range(num_epochs):
    for i in range(0,10000,batch_size):
        images = dataset[i:i + batch_size] 
        labels = labels_new[i:i + batch_size] 
        # origin shape: [100, 1, 28, 28]
        # resized: [100, 784]
        #images = images.reshape(-1, 28*28).to(device)
        labels = labels
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #if (i+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Step [{int(i/batch_size + 1)}], Loss: {loss.item():.4f}')

Epoch [1/4], Step [1], Loss: 1.8590
Epoch [1/4], Step [2], Loss: 160475.4688
Epoch [1/4], Step [3], Loss: 1415.0208
Epoch [1/4], Step [4], Loss: 142.8376
Epoch [1/4], Step [5], Loss: 15.0409
Epoch [1/4], Step [6], Loss: 18.4935
Epoch [1/4], Step [7], Loss: 1.5072
Epoch [1/4], Step [8], Loss: 2.3866
Epoch [1/4], Step [9], Loss: 1.9493
Epoch [1/4], Step [10], Loss: 1.6929
Epoch [1/4], Step [11], Loss: 7.4244
Epoch [1/4], Step [12], Loss: 1.9380
Epoch [1/4], Step [13], Loss: 1.9354
Epoch [1/4], Step [14], Loss: 1.9284
Epoch [1/4], Step [15], Loss: 1.9260
Epoch [1/4], Step [16], Loss: 1.9303
Epoch [1/4], Step [17], Loss: 2.0134
Epoch [1/4], Step [18], Loss: 1.9096
Epoch [1/4], Step [19], Loss: 1.8977
Epoch [1/4], Step [20], Loss: 1.9029
Epoch [1/4], Step [21], Loss: 1.8755
Epoch [1/4], Step [22], Loss: 1.8738
Epoch [1/4], Step [23], Loss: 1.8607
Epoch [1/4], Step [24], Loss: 1.8615
Epoch [1/4], Step [25], Loss: 1.9048
Epoch [1/4], Step [26], Loss: 1.8573
Epoch [1/4], Step [27], Loss: 1.870

KeyboardInterrupt: ignored

In [38]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for i in range(100,200, batch_size):
        #images = images.reshape(-1, 28*28).to(device)
        images = dataset[i:i + batch_size] 
        labels = labels_new[i:i + batch_size] 
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 50 test images: {acc} %')

Accuracy of the network on the 50 test images: 72.0 %


In [40]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for i in range(200,300, batch_size):
        #images = images.reshape(-1, 28*28).to(device)
        images = dataset[i:i + batch_size] 
        labels = labels_new[i:i + batch_size] 
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 50 test images: {acc} %')

Accuracy of the network on the 50 test images: 74.0 %


In [44]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for i in range(600,700, batch_size):
        #images = images.reshape(-1, 28*28).to(device)
        images = dataset[i:i + batch_size] 
        labels = labels_new[i:i + batch_size] 
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 50 test images: {acc} %')

Accuracy of the network on the 50 test images: 68.0 %


In [48]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for i in range(0,845, batch_size):
        #images = images.reshape(-1, 28*28).to(device)
        images = dataset[i:i + batch_size] 
        labels = labels_new[i:i + batch_size] 
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 50 test images: {acc} %')

Accuracy of the network on the 50 test images: 65.44378698224853 %
